## CNN Text Classification with Keras

In [1]:
import plaidml.keras
plaidml.keras.install_backend()

%run Setup.ipynb
%run ExtraFunctions.ipynb

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...
[+] Using Keras version 2.0.8
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 33856 unique tokens.
[+] Shape of data tensor: (45318, 30)
[+] Shape of label tensor: (45318, 6)
[+] Number of entries in each category:
[+] Training:
 [ 7754.  8268.  4773. 12722.  1846.   892.]
[+] Validation:
 [1890. 2002. 1256. 3220.  477.  218.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Deeper 1D CNN
[Reference](https://github.com/richliao/textClassifier), [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper Convolutional neural network: In [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

convs, filter_sizes = [], [2,3,5]
for fsz in filter_sizes:
    l_conv = Conv1D(filters=16,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(2)(l_conv)
    convs.append(l_pool)

l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(24, 3, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.4)(l_pool1)
l_cov2 = Conv1D(16, 3, activation='relu')(l_drop1)
l_pool2 = MaxPooling1D(17)(l_cov2) # global max pooling
l_flat = Flatten()(l_pool2)
l_dense = Dense(16, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

INFO:plaidml:b'Opening device "ati_radeon_pro_560_compute_engine.0"'


In [4]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

In [5]:
#tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [6]:
!rm -R logs

rm: logs: No such file or directory


In [7]:
model.summary()
model.save("cnn_kim.h5")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 30, 200)       6771400     input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 29, 16)        6416        embedding_1[0][0]                
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 28, 16)        9616        embedding_1[0][0]                
___________________________________________________________________________________________

In [9]:
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=50)

Training Progress:
Train on 36255 samples, validate on 9063 samples
Epoch 1/20
  200/36255 [..............................] - ETA: 2841s - loss: 1.6981 - acc: 0.2100

KeyboardInterrupt: 